In [12]:
from flightsql import FlightSQLClient
from dash import Dash, html, dcc
import plotly.express as px
import plotly.io as pio

import pandas as pd
import pyarrow.dataset as ds
from IPython.display import display, HTML
pio.renderers.default = "vscode"
# Read only token for demo purposes
token = "6mFPNdcEwrjQD9utxkkMS6BfmhJoMIYsHkI317EcGSCMZaTalYADf0zm6u4VqrBv5YiGvvOf5Qa8sYTVrDigeA=="
 
client = FlightSQLClient(host='eu-central-1-1.aws.cloud2.influxdata.com',
                        token=token,
                        metadata={'bucket-name': 'factory'},
                        features={'metadata-reflection': 'true'})

# Execute a query against InfluxDB's Flight SQL endpoint                         
#query = client.execute("SELECT * FROM iox.mqtt_consumer WHERE time > (NOW() - INTERVAL '1 DAY') ")

query = client.execute("SELECT * FROM iox.machine_data WHERE time > (NOW() - INTERVAL '1 DAY')")

# Create reader to consume result
reader = client.do_get(query.endpoints[0].ticket)

# Read all data into a pyarrow.Table
Table = reader.read_all()
print(Table)

pyarrow.Table
host: string
load: double
machineID: string
power: double
provider: string
temperature: double
time: timestamp[ns] not null
topic: string
vibration: double
----
host: [["285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb",...,"285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb"],["285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb",...,"285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb"],...,["285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb",...,"285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb"],["285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb",...,"285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb"]]
load: [[50,50,50,50,50,...,55,55,55,55,55],[66,66,66,66,66,...,43,43,43,43,43],...,[23,23,23,23,23,...,75,75,75,75,75],[75,75,75,75,75,...,120,120,120,120,120]]
machineID: [["machine1","mac

In [13]:
# Convert to Pandas DataFrame
df = Table.to_pandas()
df = df.sort_values(by="time")
display(df)

,host,load,machineID,power,provider,temperature,time,topic,vibration
4956,285b7993efcb,50.0,machine2,201.0,Bell PLC,38.0,2023-03-22 11:19:28.028650694,machine/machine2,83.0
0,285b7993efcb,50.0,machine1,220.0,"Arroyo, Collier and Hall",39.0,2023-03-22 11:19:28.029203039,machine/machine1,88.0
21425,285b7993efcb,50.0,machine3,220.0,Thompson PLC,36.0,2023-03-22 11:19:28.029227188,machine/machine3,82.0
4957,285b7993efcb,50.0,machine2,210.0,Bell PLC,37.0,2023-03-22 11:19:29.029073321,machine/machine2,86.0
1,285b7993efcb,50.0,machine1,205.0,"Arroyo, Collier and Hall",38.0,2023-03-22 11:19:29.029597237,machine/machine1,80.0
...,...,...,...,...,...,...,...,...,...
10405,285b7993efcb,68.0,machine2,200.0,Bell PLC,38.0,2023-03-22 15:26:08.283046828,machine/machine2,83.0
9298,285b7993efcb,6.0,machine1,199.0,"Arroyo, Collier and Hall",30.0,2023-03-22 15:26:08.535957283,machine/machine1,59.0
26382,285b7993efcb,120.0,machine3,302.0,Thompson PLC,81.0,2023-03-22 15:26:08.592972368,machine/machine3,318.0
10406,285b7993efcb,68.0,machine2,220.0,Bell PLC,35.0,2023-03-22 15:26:09.283580673,machine/machine2,81.0


In [14]:
# Create a plotly express figure (Line grah of raw Co2 Levels )
fig1 = px.line(df, x="time", y="vibration", color='machineID', title='Vibration Levels')
fig1.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed